In [1]:
#DOCUMENT CLASSIFICATION USING BERT AND TRANSFORMERS

#S.K.M.Sanjana (22BCE7474)

# Install required packages
!pip install transformers torch pandas scikit-learn tqdm

import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score, f1_score
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import pandas as pd

# For file upload in Colab
from google.colab import files

# Step 1: Upload your dataset
uploaded = files.upload()  # Select your CSV file with 'text' and 'label' columns
file_name = list(uploaded.keys())[0]

# Load dataset
df = pd.read_csv(file_name)
print("\nDataset Preview:")
print(df.head())

# Configuration
TEXT_COLUMN = "text"
LABEL_COLUMN = "label"

# Label preprocessing
label_map = {i: label for i, label in enumerate(df[LABEL_COLUMN].astype("category").cat.categories)}
df[LABEL_COLUMN] = df[LABEL_COLUMN].astype("category").cat.codes

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Fixed Dataset Class
class DocumentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Create dataset
dataset = DocumentDataset(df[TEXT_COLUMN], df[LABEL_COLUMN], tokenizer)

# Train/Validation split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoader setup
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Model initialization
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_map)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training setup
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

# Training loop
EPOCHS = 3
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{EPOCHS}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}")

# Evaluation function
def evaluate_model(model, dataloader, device, label_map):
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            preds.extend(predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average="weighted")
    report = classification_report(true_labels, preds, target_names=list(label_map.values()))

    return accuracy, f1, report

# Validation evaluation
val_accuracy, val_f1, val_report = evaluate_model(model, val_loader, device, label_map)
print("\nValidation Results:")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"F1 Score: {val_f1:.4f}")
print("Classification Report:")
print(val_report)

# Prediction function
def predict_text(model, tokenizer, text, device, label_map):
    model.eval()
    encoding = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        prediction = torch.argmax(probs, dim=-1).item()

    return label_map[prediction], probs.cpu().numpy()[0]

# Test predictions
test_samples = [
    "Invoice number 12345 for services rendered",
    "Monthly financial report Q3 2023",
    "Purchase order request for 100 units",
    "Shipping confirmation for order #7890"
]

print("\nTest Predictions:")
for sample in test_samples:
    pred_label, probabilities = predict_text(model, tokenizer, sample, device, label_map)
    print(f"\nText: {sample}")
    print(f"Predicted: {pred_label}")
    print(f"Probabilities: {probabilities}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Saving company-document-text.csv to company-document-text.csv

Dataset Preview:
                                                text          label  \
0  order id  10718 shipping details  ship name  k...  ShippingOrder   
1  invoice order id  10707 customer id  arout ord...        invoice   
2  order id  10448 shipping details  ship name  r...  ShippingOrder   
3  invoice order id  11068 customer id  queen ord...        invoice   
4  order id  10656 shipping details  ship name  g...  ShippingOrder   

   word_count  
0         120  
1          66  
2          96  
3          68  
4         109  


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Epoch 1 completed. Average Loss: 0.3814


Training Epoch 2/3: 100%|██████████| 67/67 [46:41<00:00, 41.81s/it]


Epoch 2 completed. Average Loss: 0.0344


Training Epoch 3/3: 100%|██████████| 67/67 [46:41<00:00, 41.82s/it]


Epoch 3 completed. Average Loss: 0.0108

Validation Results:
Accuracy: 1.0000
F1 Score: 1.0000
Classification Report:
                precision    recall  f1-score   support

 ShippingOrder       1.00      1.00      1.00       173
       invoice       1.00      1.00      1.00       163
purchase Order       1.00      1.00      1.00       162
        report       1.00      1.00      1.00        38

      accuracy                           1.00       536
     macro avg       1.00      1.00      1.00       536
  weighted avg       1.00      1.00      1.00       536


Test Predictions:

Text: Invoice number 12345 for services rendered
Predicted: purchase Order
Probabilities: [0.09937808 0.1583935  0.5840553  0.15817314]

Text: Monthly financial report Q3 2023
Predicted: purchase Order
Probabilities: [0.0199279  0.11074626 0.7156536  0.15367226]

Text: Purchase order request for 100 units
Predicted: purchase Order
Probabilities: [0.01484638 0.01010433 0.9497017  0.02534752]

Text: Shipping c